<a href="https://colab.research.google.com/github/GabHoo/Challenging-SRL/blob/main/Allen_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install allennlp
pip install allennlp-models
pip install -U spaCy
pip install checklist


In [ ]:
from allennlp.predictors import Predictor
import allennlp_models.tagging

import json

from google.colab import drive
drive.mount('/content/gdrive/')

!cp gdrive/MyDrive/Challenging-SRL/utils.py .
from utils import *

ModuleNotFoundError: ignored

In [ ]:
predictor_Bert = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")

predictor_Bilstm = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#CREATING DATA SANDBOX

In [ ]:
import checklist
from checklist.editor import Editor
import numpy as np
from checklist.perturb import Perturb

In [ ]:
editor = Editor()


In [ ]:
ret = editor.template('This is a {adj} movie.', adj=['good', 'great', 'awesome', 'excellent'])

ret.data

['This is a good movie.',
 'This is a great movie.',
 'This is a awesome movie.',
 'This is a excellent movie.']

###Creation semi manual of the contraction data test
Checklist will contract or expand the sentence. Here are some initial predicates that can be subjected to this phenomena

In [ ]:
data = ['it\'s a wonderfull day',
        "There's some pesto left",
        "I shoud have tried that as well",#!
        "They will leave the house",
        "That would be creazy",
        "we are here",
        'Mark had not see that coming',
        'She will be a great candidate',
        'Mary is not a nurse.',
        'He\'s gone already',
        "I would like some tea",
         "I would say something to them",
        "I could not eat some food now",
        'We\'ve decided to change house']

In [ ]:
ret = Perturb.perturb(data, Perturb.contractions,keep_original=True)

In [ ]:
ret.data #save into a file

[["it's a wonderfull day", 'it is a wonderfull day'],
 ["There's some pesto left", 'There is some pesto left'],
 ['They will leave the house', "They'll leave the house"],
 ['That would be creazy', "That'd be creazy"],
 ['we are here', "we're here"],
 ['Mark had not see that coming', "Mark hadn't see that coming"],
 ['She will be a great candidate', "She'll be a great candidate"],
 ['Mary is not a nurse.', "Mary isn't a nurse."],
 ["He's gone already", 'He is gone already'],
 ['I would like some tea', "I'd like some tea"],
 ['I would say something to them', "I'd say something to them"],
 ['I could not eat some food now', "I couldn't eat some food now"],
 ["We've decided to change house", 'We have decided to change house']]

In [ ]:
def evaluate_PI_INV_contractions(predictor,data):
  """This function returns a failure rate (percentage) that represents how many coupled sentnece 
  presents inconsitnency in their tags of a contracted predicate. The idea is that contracting a verb should not cause any changes! 
  Matching is done as following: tags need to be exactly the same thus the perturbation does not have to introduce
  any more (or less) token or change the arguments. This checks only fits very few purposes.
  :predictor
  :data is a list of lists where the nested lists are couple of sentence to be compared"""
  fail=0
  for s in sentences:
    s1,s2=s
    pred1=predictor.predict(s1)
    pred2=predictor.predict(s2)

    v1=get_identified_verbs(pred1)
    v2=get_identified_verbs(pred2)
    
    if len(v1)!=len(v2):
      print(f"Failure in {s1} | {s2}")
      print(f"Different number of verbs found: {v1=} against {v2=}")
      fail+=1
      continue
    
    for v1,v2 in zip(pred1['verbs'],pred2['verbs']):
      if v1['verb']!=v2['verb']: #we only compare the verbs that are not matching exactly (aka the contracted vs original)
         if v1['tags']!=v2['tags']:
           fail+=1
           print(f"Failure in {s1} | {s2}")
           print(f"The alteration of verb {v1['verb']}/{v2['verb']} lead to a different labeling:\n")
           print(v1)
           print(v2)
           break
      
  return fail/len(data)*100


## LOOKING DATA (propBank)

In [ ]:
import json
import random

In [ ]:
f=open('data_train.jsonl')

In [ ]:
di=json.load(f)

In [ ]:
di = di["data"]
len(di)

40592

[{'seq_words': ['[',
   'This',
   'killing',
   'of',
   'a',
   'respected',
   'cleric',
   'will',
   'be',
   'causing',
   'us',
   'trouble',
   'for',
   'years',
   'to',
   'come',
   '.',
   ']'],
  'BIO': ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'ARG1',
   'O',
   'V',
   'O',
   'O'],
  'pred_sense': [15, 'come.01']},
 {'seq_words': ['[',
   'This',
   'killing',
   'of',
   'a',
   'respected',
   'cleric',
   'will',
   'be',
   'causing',
   'us',
   'trouble',
   'for',
   'years',
   'to',
   'come',
   '.',
   ']'],
  'BIO': ['O',
   'O',
   'ARG0',
   'O',
   'O',
   'O',
   'O',
   'ARGM-MOD',
   'O',
   'V',
   'ARGM-GOL',
   'ARG1',
   'O',
   'ARGM-TMP',
   'O',
   'O',
   'O',
   'O'],
  'pred_sense': [9, 'cause.01']},
 {'seq_words': ['[',
   'This',
   'killing',
   'of',
   'a',
   'respected',
   'cleric',
   'will',
   'be',
   'causing',
   'us',
   'trouble',
   'for',
   'years',
   'to',
  

In [ ]:
[(' '.join(i['seq_words']),i['BIO']) for i in di[0:6]]

In [ ]:
prediction=predictor.predict(
    sentence="This killing of a respected cleric will be causing us trouble for years to come "
)
prediction_lstm=predictor_bilstm.predict(
    sentence="This killing of a respected cleric will be causing us trouble for years to come "
)

In [ ]:
prediction

In [ ]:
prediction_lstm


#PREDICATE IDENTIFICATION

##VOCABULARY

###Test for contractions

In [ ]:
fail_bilstm=evaluate_PI_INV_contractions(predictor_Bilstm,data)

NameError: ignored

**bold text**##Abrreviation

###Test for irregular inflections

##AMBIGUITY

###Experiment with words potentially be verbs [POLISEMIC]

In [ ]:
print("Tesiting Polysemic words behaviour (DIR) in Preicate Identification\n")

f = open('/content/gdrive/MyDrive/Challenging-SRL/polysem_verbs_DIR.json')
data = json.load(f)

print("Bart model:")
rate_Bart=evaluate_verbIdentification_DIR(predictor_Bart,data)
print(f"Failure rate {rate_Bart}\n")

print("Bisltm model:")
rate_bilstm=evaluate_verbIdentification_DIR(predictor_Bilstm,data)
print(f"Failure rate {rate_bilstm}\n")

Tesiting Polysemic words behaviour (DIR) in Preicate Identification

Bart model:
Failed for: [I always turn left at the stop sign.],[She left her phone at home.]. left in both did NOT chage the predicate identification
['turn', 'left'] ['left']


Failed for: [I need help as soon as possible],[Food is a vital need]. need in both did NOT chage the predicate identification
['need'] ['is']


Failure rate 6.666666666666667

Bisltm model:
Failed for: [I always turn left at the stop sign.],[She left her phone at home.]. left in both did NOT chage the predicate identification
['turn', 'left'] ['left']


Failed for: [I need help as soon as possible],[Food is a vital need]. need in both did NOT chage the predicate identification
['need'] ['is']


Failure rate 6.666666666666667



'I also had a pedicure and they cut my nails too short one of my big toes looks like it s getting infected .'

###Experiment with verbs being in different roles -ing

In [ ]:
sentences = [    "The writing on the wall was a warning of what was to come.",    "The painting of the sunset was breathtakingly beautiful.",    "The singing of the birds was a welcome sound in the morning.","The singing of the birds was a welcome sound in the morning."    "The reading of the will revealed some surprising information.",    "The dancing of the flames mesmerized the audience.",    "The running of the river was soothing to listen to.",    "The howling of the wind kept me awake at night.",    "The barking of the dog alerted us to the intruder.",    "The playing of the piano filled the room with music.",    "The cleaning of the house was a tedious chore.",    "The mowing of the lawn was a necessary task.",    "The washing of the dishes was a never-ending job.",    "The painting of the house was a weekend project.",    "The cooking of the meal took longer than expected.",    "The building of the bridge was a remarkable feat of engineering.",    "The driving of the car was a peaceful way to clear my mind.",    "The typing of the report was interrupted by a power outage.",    "The repairing of the bike took longer than anticipated.",    "The cutting of the cake was the highlight of the party.",    "The playing of the game was a fun way to spend the afternoon."]


In [ ]:
sentences = [    "I have always enjoyed cooking, it is one of my favourite hobbies.",    "The protests that followed the killing of the priest were widespread.",    "The sound of the river running was soothing to listen to.",    "The information revealed in the reading of the will was surprising.",    "The audience was mesmerized by the dancing flames.",    "The sunset was breathtakingly beautiful in the painting.",    "The birds singing in the morning was a welcome sound.",    "The intruder was alerted by the barking of the dog.",    "The music filled the room with the playing of the piano.",    "The tedious chore of cleaning the house took all day.",    "The necessary task of mowing the lawn was finally finished.",    "The never-ending job of washing the dishes always seemed to pile up.",    "The remarkable feat of engineering was achieved in the building of the bridge.",    "The peaceful way to clear my mind was through driving the car.",    "The power outage interrupted the typing of the report.",    "The longer than anticipated repair of the bike was finally completed.",    "The highlight of the party was cutting the cake.",    "The fun way to spend the afternoon was playing the game.",    "The weekend project of painting the house was finally done.",    "The wind howling kept me awake at night.",]


can you create 5 example sentence where there is a verb not in the semantic role of the verb such as: "cooking is one of my favourite hobbies", or "the killing of the priest sparked a lot of protests"


In [ ]:
#se=[ "The birds singing was a welcome sound in the morning.", "The birds's singing was a welcome sound in the morning.", "The singing of the birds was a welcome sound in the morning." ]

se=["cooking is what I love","I love cooking ","I love cooking in the morning", "I loved cooking with you","I have always liked cooking","cooking is a great hooby","cooking vegetables is a great hobby"]

In [ ]:
for s in se:
  print(s)
  pred=predictor.predict(s)
  
  verbs=[x['verb'] for x in pred['verbs']]
  #print(pred)
  print(verbs)
  #print(pred)

cooking is what I love
['is', 'love']
I love cooking 
['love', 'cooking']
I love cooking in the morning
['love', 'cooking']
I loved cooking with you
['loved', 'cooking']
I have always liked cooking
['have', 'liked', 'cooking']
cooking is a great hooby
['is']
cooking vegetables is a great hobby
['cooking', 'is']


In [ ]:
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  verbs=[x['verb'] for x in pred['verbs']]
  print(verbs)
  #print(pred)

I have always enjoyed cooking, it is one of my favourite hobbies.
['have', 'enjoyed', 'is']
The protests that followed the killing of the priest were widespread.
['followed', 'were']
The sound of the river running was soothing to listen to.
['running', 'was', 'soothing', 'listen']
The information revealed in the reading of the will was surprising.
['revealed', 'was']
The audience was mesmerized by the dancing flames.
['was', 'mesmerized']
The sunset was breathtakingly beautiful in the painting.
['was']
The birds singing in the morning was a welcome sound.
['singing', 'was']
The intruder was alerted by the barking of the dog.
['was', 'alerted']
The music filled the room with the playing of the piano.
['filled']
The tedious chore of cleaning the house took all day.
['cleaning', 'took']
The necessary task of mowing the lawn was finally finished.
['mowing', 'was', 'finished']
The never-ending job of washing the dishes always seemed to pile up.
['ending', 'washing', 'seemed', 'pile']
The re

In [ ]:
prediction=predictor.predict(
    sentence="I was writing a thesis"
)

In [ ]:
prediction

{'verbs': [{'verb': 'was',
   'description': 'I [V: was] writing a thesis',
   'tags': ['O', 'B-V', 'O', 'O', 'O']},
  {'verb': 'writing',
   'description': '[ARG0: I] was [V: writing] [ARG1: a thesis]',
   'tags': ['B-ARG0', 'O', 'B-V', 'B-ARG1', 'I-ARG1']}],
 'words': ['I', 'was', 'writing', 'a', 'thesis']}

In [ ]:
from nltk.corpus import propbank

In [ ]:
nltk.download('propbank')

[nltk_data] Downloading package propbank to /root/nltk_data...


True

In [ ]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [ ]:
pb_instances = propbank.instances()

In [ ]:
inst=pb_instances[103]

In [ ]:
 import nltk.corpus.reader.propbank as pb

In [ ]:
turn_01 = propbank.roleset('turn.01')

In [ ]:
for role in turn_01.findall("roles/role"):
  print(role.attrib['n'], role.attrib['descr'])

0 turner
1 thing turning
m direction, location


In [ ]:
sentences=["The birds singing in the morning was a welcome sound.","The singing of birds in the morning was a welcome sound."]
for s in sentences:
  print(s)
  predBert=predictor.predict(s)
  predBilstm=predictor_bilstm.predict(s)
  print("BERT")
  print([x['description'] for x in predBert['verbs']])

  print("Lstm")
  print([x['description'] for x in predBilstm['verbs']])
  
  print("\n\n")

The birds singing in the morning was a welcome sound.
BERT
['[ARG0: The birds] [V: singing] [ARGM-TMP: in the morning] was a welcome sound .', '[ARG1: The birds singing in the morning] [V: was] [ARG2: a welcome sound] .']
Lstm
['[ARG0: The birds] [V: singing] [ARGM-TMP: in the morning] was a welcome sound .', '[ARG1: The birds singing in the morning] [V: was] [ARG2: a welcome sound] .']



The singing of birds in the morning was a welcome sound.
BERT
['[ARG1: The singing of birds in the morning] [V: was] [ARG2: a welcome sound] .']
Lstm
['[ARG1: The singing of birds in the morning] [V: was] [ARG2: a welcome sound] .']





In [ ]:
sentences=["It's running low ","It is running low"]
for s in sentences:
  print(s)
  predBert=predictor.predict(s)
  predBilstm=predictor_bilstm.predict(s)
  print("BERT")
  print([x['description'] for x in predBert['verbs']])

  print("Lstm")
  print([x['description'] for x in predBilstm['verbs']])
  
  print("\n\n")

It's running low 
BERT
["It [V: 's] running low", "[ARG0: It] 's [V: running] [ARGM-MNR: low]"]
Lstm
["It [V: 's] running low", "[ARG1: It] 's [V: running] [ARG2: low]"]



It is running low
BERT
['It [V: is] running low', '[ARG1: It] is [V: running] [ARGM-MNR: low]']
Lstm
['[ARG1: It] [V: is] [ARG2: running low]', '[ARG1: It] is [V: running] [ARG2: low]']





**bold text**#FOR ARGOUMENT CLASSIFICATION

##**RARITY** or LOW SHOT 

###SLANG VEBRS

In [ ]:
predictor_Bert.predict("I really want to do it"),predictor_Bert.predict("I really wanna do it")
predictor_Bert.predict("I am gonna do it"),predictor_Bert.predict("I am going to do it")
[gimmie - give me, ]

({'verbs': [{'verb': 'want',
    'description': '[ARG0: I] [ARGM-ADV: really] [V: want] [ARG1: to do it]',
    'tags': ['B-ARG0', 'B-ARGM-ADV', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1']},
   {'verb': 'do',
    'description': '[ARG0: I] really want to [V: do] [ARG1: it]',
    'tags': ['B-ARG0', 'O', 'O', 'O', 'B-V', 'B-ARG1']}],
  'words': ['I', 'really', 'want', 'to', 'do', 'it']},
 {'verbs': [{'verb': 'wanna',
    'description': '[ARG0: I] [ARGM-ADV: really] [V: wanna] [ARG1: do it]',
    'tags': ['B-ARG0', 'B-ARGM-ADV', 'B-V', 'B-ARG1', 'I-ARG1']},
   {'verb': 'do',
    'description': '[ARG0: I] really wanna [V: do] [ARG1: it]',
    'tags': ['B-ARG0', 'O', 'O', 'B-V', 'B-ARG1']}],
  'words': ['I', 'really', 'wanna', 'do', 'it']})

### MODERN VERBS

#AROUGMENTS CLASSIFICATION

##VOCABULAIRTY+POS

###Enrtity flipping

In [ ]:
sent1 = "I drink some whisky with Jhon "
sent2 = "I drink some whisky with ice "

In [ ]:
pred1=predictor.predict(
    sentence=sent1
)
pred2=predictor.predict(
    sentence=sent2
)

In [ ]:
pred1

{'verbs': [{'verb': 'drink',
   'description': '[ARG0: I] [V: drink] [ARG1: some whisky] [ARGM-COM: with Jhon]',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-COM', 'I-ARGM-COM']}],
 'words': ['I', 'drink', 'some', 'whisky', 'with', 'Jhon']}

In [ ]:
pred2

{'verbs': [{'verb': 'drink',
   'description': '[ARG0: I] [V: drink] [ARG1: some whisky] [ARGM-MNR: with ice]',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-MNR', 'I-ARGM-MNR']}],
 'words': ['I', 'drink', 'some', 'whisky', 'with', 'ice']}

###Pronouns vs Entity

In [ ]:
def make_tag_list(v): #v=pred['verb'][i]
  matches=re.findall(r'\[(.*?)\]', v['description'])
  tags=[x.split(":")[0] for x in matches]
  return tags

In [ ]:
def validate_INV_alltags(pred1,pred2):
  #assert pred1['words']==pred2['words'],f"ERROR, Comparing two different sentences"
  if len(pred1['verbs'])!=len(pred2['verbs']):
    print("not the same predicates were found ")
    print(pred1['words'],pred2['words'])
    print([x['verb'] for x in pred1['verbs']],[x['verb'] for x in pred2['verbs']],"\n")
    return False

  for v in zip(pred1['verbs'],pred2['verbs']): 
      v1,v2=v
      
      if v1['verb']!=v2['verb']:
        print("missmatch in verb identification \n,v1['verb'] and v2['verb']\n")
        return False
      
      tag1=make_tag_list(v1)

      tag2=make_tag_list(v2)

      #if v1['tags']!=v2['tags']:
      if tag1!=tag2:
        print(f"missmatch for tags of verb {v1['verb']}\n{v1['description']}\n{v2['description']}\n")
        return False

      continue
  return True

In [ ]:
sentences=[    [        "The cat sat on the mat.",         "It sat on the mat."    ], 
    [        "John gave Mary the book.",         "He gave her the book."    ], 
    [        "The sun rises in the east.",         "It rises in the east."    ],
    [        "The dog chased the cat and caught her.",         "It chased her and caught her."    ],
    [        "The students submitted their homework on time.",         "They submitted it on time."    ],
    [        "The cake was baked by my mom.",         "It was baked by her."    ],
    [        "The company hired a new employee.",         "It hired them."    ],
    [        "The baby cried and his mother picked him up.",         "It cried and she picked it up."    ],
    [        "The train arrived at the station late.",         "It arrived at the station late."    ],
    [        "The tree was blown over by the wind.",         "It was blown over by it."    ],
    [        "The waiter brought the food to the table.",         "He brought it to the table."    ],
    [        "The doctor examined the patient and prescribed medication.",         "They examined them and prescribed it."    ],
    [        "The children played in the park all day.",         "They played in it all day."    ],
    [        "The author wrote a best-selling novel.",         "They wrote a best-selling novel."    ],
    [        "The car hit the tree and damaged it.",         "It hit it and damaged it."    ],
    [        "The teacher graded the tests and returned them to the students.",         "They graded them and returned them to them."    ],
    [        "The phone rang and the receptionist answered it.",         "It rang and they answered it."    ],
    [        "The chef prepared the meal and served it to the customers.",         "They prepared it and served it to them."    ],
    [        "The athlete broke the world record.",         "They broke it."    ],
    [        "The artist painted a beautiful portrait.",         "They painted a beautiful portrait."    ]
]


In [ ]:
for p in sentences:
  
  active,passive=p

  print("BERT")
  predBert_a = predictor.predict(active)
  predBert_p = predictor.predict(passive) 
  
  print([x['description'] for x in predBert_a['verbs']])
  print([x['description'] for x in predBert_p['verbs']])
  print("\n")

print("\n\n")
for p in sentences:

  active,passive=p

  print("Lstm")
  predBilstm_a=predictor_bilstm.predict(active)
  predBilstm_p=predictor_bilstm.predict(passive)
  print([x['description'] for x in predBilstm_a['verbs']])
  print([x['description'] for x in predBilstm_p['verbs']])
  print("\n\n")



BERT
['[ARG1: The cat] [V: sat] [ARG2: on the mat] .']
['[ARG1: It] [V: sat] [ARG2: on the mat] .']


BERT
['[ARG0: John] [V: gave] [ARG2: Mary] [ARG1: the book] .']
['[ARG0: He] [V: gave] [ARG2: her] [ARG1: the book] .']


BERT
['[ARG1: The sun] [V: rises] [ARGM-LOC: in the east] .']
['[ARG1: It] [V: rises] [ARGM-LOC: in the east] .']


BERT
['[ARG0: The dog] [V: chased] [ARG1: the cat] and caught her .', '[ARG0: The dog] chased the cat and [V: caught] [ARG1: her] .']
['[ARG0: It] [V: chased] [ARG1: her] and caught her .', '[ARG0: It] chased her and [V: caught] [ARG1: her] .']


BERT
['[ARG0: The students] [V: submitted] [ARG1: their homework] [ARGM-TMP: on time] .']
['[ARG0: They] [V: submitted] [ARG1: it] [ARGM-TMP: on time] .']


BERT
['The cake [V: was] baked by my mom .', '[ARG1: The cake] was [V: baked] [ARG0: by my mom] .']
['It [V: was] baked by her .', '[ARG1: It] was [V: baked] [ARG0: by her] .']


BERT


KeyboardInterrupt: ignored

In [ ]:
sentences=[    [        "The doctor prescribed medication for the patient's headache.",         "They prescribed it for their headache."    ],
    [        "The company's CEO announced a new product launch during the conference.",         "They announced it during the conference."    ],
    [        "The teacher explained the complex math problem to the students in detail.",         "They explained it to them in detail."    ],
    [        "The committee reviewed the proposal and made several recommendations for improvement.",         "They reviewed it and made several recommendations for improvement."    ],
    [        "The company hired a new employee with extensive experience in the field.",         "It hired them with extensive experience in the field."    ],
    [        "The chef used fresh herbs and spices to add flavor to the dish.",         "They used them to add flavor to it."    ],
    [        "The lawyer defended his client against the false allegations.",         "They defended them against them."    ],
    [        "The student completed the research project and submitted it before the deadline.",         "They completed it and submitted it before it."    ],
    [        "The artist created a stunning sculpture using a variety of materials.",         "They created it using a variety of materials."    ],
    [        "The scientist conducted a series of experiments to test the hypothesis.",         "They conducted them to test it."    ],
    [        "The athlete trained for months before competing in the Olympic Games.",         "They trained for months before competing in them."    ],
    [        "The architect designed the building to be energy efficient and environmentally friendly.",         "They designed it to be energy efficient and environmentally friendly."    ],
    [        "The writer crafted a compelling storyline with well-developed characters.",         "They crafted it with them."    ],
    [        "The company's marketing team created a new advertising campaign to promote the product.",         "They created it to promote it."    ],
    [        "The nurse administered the medication to the patient and monitored their condition.",         "They administered it to them and monitored their condition."    ],
    [        "The technician repaired the computer and installed new software.",         "They repaired it and installed it."    ],
    [        "The musician composed a beautiful melody that moved the audience to tears.",         "They composed it and moved them to tears."    ],
    [        "The sales team negotiated a deal with the client and closed the sale.",         "They negotiated it with them and closed it."    ],
    [        "The construction crew built a new bridge to improve traffic flow.",         "They built it to improve it."    ],
    [        "The therapist helped the patient develop coping mechanisms to manage their anxiety.",         "They helped them develop them to manage it."    ]
]


In [ ]:
def eval_INV_proNounes_entities(sentences,predictor):
  fails=0
  for p in sentences:
    
    ent,pronoun=p

    predBert_e = predictor.predict(ent)
    predBert_p = predictor.predict(pronoun) 
    
    #print([x['description'] for x in predBert_e['verbs']])
    #print([x['description'] for x in predBert_p['verbs']])

    if not (validate_INV_alltags(predBert_e,predBert_p)):
      
      fails+=1

  return (fails/len(sentences)*100)

bert_failure=eval_INV_proNounes_entities(sentences,predictor)
print()
"""
print("\n\n")
for p in sentences:

  active,passive=p

  print("Lstm")
  predBilstm_a=predictor_bilstm.predict(active)
  predBilstm_p=predictor_bilstm.predict(passive)
  print([x['description'] for x in predBilstm_a['verbs']])
  print([x['description'] for x in predBilstm_p['verbs']])
  print("\n\n")
"""

missmatch for tags of verb blown
[ARG1: The tree] was [V: blown] [ARGM-DIR: over] [ARG0: by the wind] .
[ARG1: It] was [V: blown] over [ARG0: by it] .

not the same predicates were found 
['The', 'doctor', 'examined', 'the', 'patient', 'and', 'prescribed', 'medication', '.'] ['They', 'examined', 'them', 'and', 'prescribed', 'it', '.']
['examined'] ['examined', 'prescribed'] 

not the same predicates were found 
['The', 'phone', 'rang', 'and', 'the', 'receptionist', 'answered', 'it', '.'] ['It', 'rang', 'and', 'they', 'answered', 'it', '.']
['answered'] ['rang', 'answered'] 



'\nprint("\n\n")\nfor p in sentences:\n\n  active,passive=p\n\n  print("Lstm")\n  predBilstm_a=predictor_bilstm.predict(active)\n  predBilstm_p=predictor_bilstm.predict(passive)\n  print([x[\'description\'] for x in predBilstm_a[\'verbs\']])\n  print([x[\'description\'] for x in predBilstm_p[\'verbs\']])\n  print("\n\n")\n'

In [ ]:
bert_failure


15.0

3

In [ ]:
pred1=predictor.predict("The athletes trained for months before competing")
pred2=predictor.predict("They trained for  months before competing")

In [ ]:
pred1=predictor.predict("They eat an apple")
pred2=predictor_bilstm.predict("They eat an apple")

In [ ]:
a1= "The baby cried and his mother picked him up." 
a2= "It cried and she picked it up." 
pred1=predictor.predict(a1)
pred2=predictor.predict(a2)

In [ ]:
pred1,pred2

({'verbs': [{'verb': 'cried',
    'description': '[ARG0: The baby] [V: cried] and his mother picked him up .',
    'tags': ['B-ARG0', 'I-ARG0', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O']},
   {'verb': 'picked',
    'description': 'The baby cried and [ARG0: his mother] [V: picked] [ARG1: him] up .',
    'tags': ['O',
     'O',
     'O',
     'O',
     'B-ARG0',
     'I-ARG0',
     'B-V',
     'B-ARG1',
     'O',
     'O']}],
  'words': ['The',
   'baby',
   'cried',
   'and',
   'his',
   'mother',
   'picked',
   'him',
   'up',
   '.']},
 {'verbs': [{'verb': 'cried',
    'description': '[ARG0: It] [V: cried] and she picked it up .',
    'tags': ['B-ARG0', 'B-V', 'O', 'O', 'O', 'O', 'O', 'O']},
   {'verb': 'picked',
    'description': 'It cried and [ARG0: she] [V: picked] [ARG1: it] up .',
    'tags': ['O', 'O', 'O', 'B-ARG0', 'B-V', 'B-ARG1', 'O', 'O']}],
  'words': ['It', 'cried', 'and', 'she', 'picked', 'it', 'up', '.']})

'[ARG0: The baby] [V: cried] and his mother picked him up .'

In [ ]:
import re
text=pred1['verbs'][0]['description']
matches = re.findall(r'\[(.*?)\]', text)
matches

['ARG0: The baby', 'V: cried']

In [ ]:
def make_tag_list(v): #v=pred['verb'][i]
  matches=re.findall(r'\[(.*?)\]', v['description'])
  tags=[x.split(":")[0] for x in matches]
  return tags

In [ ]:
make_tag_list(pred1['verbs'][0])

['ARG0', 'V']

##Robustness

###word orders [overlapping spans]


In [ ]:
sentences=["the coach teaches to the team coordination", "The coach teaches coordination to the team"]
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

the coach teaches to the team coordination
['[ARG0: the coach] [V: teaches] [ARG2: to the team coordination]']



The coach teaches coordination to the team
['[ARG0: The coach] [V: teaches] [ARG1: coordination to the team]']





In [ ]:
sentences=["I like that mark has not beeing able to reduce sigarets despite his mother dying of lung cancer"]
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

I like that mark has not beeing able to reduce sigarets despite his mother dying of lung cancer
['[ARG0: I] [V: like] [ARG1: that mark has not beeing able to reduce sigarets despite his mother dying of lung cancer]', 'I like that mark [V: has] not beeing able to reduce sigarets despite his mother dying of lung cancer', 'I like that [ARG1: mark] has [ARGM-NEG: not] [V: beeing] [ARGM-PRD: able to reduce sigarets] [ARGM-ADV: despite his mother dying of lung cancer]', 'I like that [ARG0: mark] has not beeing able to [V: reduce] [ARG1: sigarets] despite his mother dying of lung cancer', 'I like that mark has not beeing able to reduce sigarets despite [ARG1: his mother] [V: dying] [ARGM-CAU: of lung cancer]']





In [ ]:
sentences=["I give to you kisses", "I give kisses to you"]
for s in sentences:
  print(s)
  pred=predictor_bilstm.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

I give to you kisses
['[ARG0: I] [V: give] [ARG2: to you kisses]']



I give kisses to you
['[ARG0: I] [V: give] [ARG1: kisses] [ARG2: to you]']





In [ ]:
sentences=["you are stupid"]
for s in sentences:
  print(s)
  
  pred=predictor_lstm.predict(s)
  print(pred)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

you are stupid


NameError: ignored

In [ ]:
sentences=[" looking eating kisses Robert at lunchtime in the park"]
for s in sentences:
  print(s)
  predBert=predictor.predict(s)
  predBilstm=predictor_bilstm.predict(s)
  print("BERT")
  print([x['description'] for x in predBert['verbs']])

  print("Lstm")
  print([x['description'] for x in predBilstm['verbs']])
  
  print("\n\n")

 looking eating kisses Robert at lunchtime in the park
BERT
['[V: looking] [ARG1: eating kisses Robert at lunchtime in the park]', 'looking [V: eating] [ARG1: kisses Robert] [ARGM-TMP: at lunchtime] [ARGM-LOC: in] [ARGM-TMP: the park]']
Lstm
['[V: looking] [ARG1: eating kisses Robert at lunchtime in the park]', 'looking [V: eating] [ARG1: kisses Robert at lunchtime] [ARGM-LOC: in the park]']





In [ ]:
##

In [ ]:
sentences=["I give to you love", "I give love to you"]
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

I give to you love
['[ARG0: I] [V: give] [ARG2: to you love]', 'I give to [ARG0: you] [V: love]']



I give love to you
['[ARG0: I] [V: give] [ARG1: love] [ARG2: to you]']





###passive and actrive trasnformarion

In [ ]:
paired_sentences = [    ['The teacher graded the exams.', 'The exams were graded by the teacher.'],
    ['The company launched a new product.', 'A new product was launched by the company.'],
    ['The chef prepared the meal.', 'The meal was prepared by the chef.'],
    ['The storm destroyed the house.', 'The house was destroyed by the storm.'],
    ['The doctor cured the patient.', 'The patient was cured by the doctor.'],
    ['The police caught the thief.', 'The thief was caught by the police.'],
    ['The singer performed the song.', 'The song was performed by the singer.'],
    ['The painter created a masterpiece.', 'A masterpiece was created by the painter.'],
    ['The company awarded the contract.', 'The contract was awarded by the company.'],
    ['The author wrote the book.', 'The book was written by the author.'],
    ['The team won the game.', 'The game was won by the team.'],
    ['The chef cooked the meal.', 'The meal was cooked by the chef.'],
    ['The audience applauded the performer.', 'The performer was applauded by the audience.'],
    ['The company hired a new employee.', 'A new employee was hired by the company.'],
    ['The artist painted the portrait.', 'The portrait was painted by the artist.'],
    ['The company produced a new film.', 'A new film was produced by the company.'],
    ['The doctor performed the surgery.', 'The surgery was performed by the doctor.'],
    ['The company launched a marketing campaign.', 'A marketing campaign was launched by the company.'],
    ['The waiter served the meal.', 'The meal was served by the waiter.'],
    ['The engineer designed the building.', 'The building was designed by the engineer.']
]


In [ ]:
for p in paired_sentences:
  
  active,passive=p

  print("BERT")
  predBert_a = predictor.predict(active)
  predBert_p = predictor.predict(passive) 
  
  print([x['description'] for x in predBert_a['verbs']])
  print([x['description'] for x in predBert_p['verbs']])
  print("\n")

  print("Lstm")
  predBilstm_a=predictor_bilstm.predict(active)
  predBilstm_p=predictor_bilstm.predict(passive)
  print([x['description'] for x in predBilstm_a['verbs']])
  print([x['description'] for x in predBilstm_p['verbs']])
  print("\n\n")


  print("\n\n")

BERT
['[ARG0: The teacher] [V: graded] [ARG1: the exams] .']
['The exams [V: were] graded by the teacher .', '[ARG1: The exams] were [V: graded] [ARG0: by the teacher] .']


Lstm
['[ARG0: The teacher] [V: graded] [ARG1: the exams] .']
['The exams [V: were] graded by the teacher .', '[ARG2: The exams] were [V: graded] [ARG0: by the teacher] .']






BERT
['[ARG0: The company] [V: launched] [ARG1: a new product] .']
['A new product [V: was] launched by the company .', '[ARG1: A new product] was [V: launched] [ARG0: by the company] .']


Lstm
['[ARG0: The company] [V: launched] [ARG1: a new product] .']
['A new product [V: was] launched by the company .', '[ARG1: A new product] was [V: launched] [ARG0: by the company] .']






BERT
['[ARG0: The chef] [V: prepared] [ARG1: the meal] .']
['The meal [V: was] prepared by the chef .', '[ARG1: The meal] was [V: prepared] [ARG0: by the chef] .']


Lstm
['[ARG0: The chef] [V: prepared] [ARG1: the meal] .']
['The meal [V: was] prepared by the che

In [ ]:
paired_sentences = [
    ['The consultant recommended that the board approve the merger.', 'It was recommended by the consultant that the merger be approved by the board.'],
    ['The coach trained the team on proper technique and strategy.', 'The team was trained on proper technique and strategy by the coach.'],
    ['The lawyer defended her client against charges of fraud and embezzlement.', 'The client was defended against charges of fraud and embezzlement by the lawyer.']
]


In [ ]:
for p in paired_sentences:
  
  active,passive=p

  print("BERT")
  predBert_a = predictor.predict(active)
  predBert_p = predictor.predict(passive) 
  
  print([x['description'] for x in predBert_a['verbs']])
  print([x['description'] for x in predBert_p['verbs']])
  print("\n")

  print("Lstm")
  predBilstm_a=predictor_bilstm.predict(active)
  predBilstm_p=predictor_bilstm.predict(passive)
  print([x['description'] for x in predBilstm_a['verbs']])
  print([x['description'] for x in predBilstm_p['verbs']])
  print("\n\n")


  print("\n\n")

BERT
['[ARG0: The consultant] [V: recommended] [ARG1: that the board approve the merger] .', 'The consultant recommended that [ARG0: the board] [V: approve] [ARG1: the merger] .']
['It [V: was] recommended by the consultant that the merger be approved by the board .', 'It was [V: recommended] [ARG0: by the consultant] [ARG1: that the merger be approved by the board] .', 'It was recommended by the consultant that the merger [V: be] approved by the board .', 'It was recommended by the consultant that [ARG1: the merger] be [V: approved] [ARG0: by the board] .']


Lstm
['[ARG0: The consultant] [V: recommended] [ARG1: that the board approve the merger] .', 'The consultant recommended that [ARG0: the board] [V: approve] [ARG1: the merger] .']
['It [V: was] recommended by the consultant that the merger be approved by the board .', '[ARG1: It] was [V: recommended] [ARG0: by the consultant] [ARG2: that the merger be approved by the board] .', 'It was recommended by the consultant that the merge

the coach teaches the team a new schema
['[ARG0: the coach] [V: teaches] [ARG2: the team] [ARG1: a new schema]']



The coach teaches a new schema to the team
['[ARG0: The coach] [V: teaches] [ARG1: a new schema] [ARG2: to the team]']





### DIRECT SPEECH

##AMBIGUITY

### PP -ambiguity
 

In [ ]:
sentences="I went to the resturant by the Hutsin", "I went to the resturant by"
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

I went to the resturant by the Hutsin
['[ARG0: I] [V: went] [ARG4: to the resturant by the Hutsin]']



I went to the resturant by bike
['[ARG0: I] [V: went] [ARG4: to the resturant] [ARGM-MNR: by bike]']





In [ ]:
["I fixed the car with a red logo","I fixed the car with a wretch"]
["I bought a computer with GPU"," I bought a computer wit bitcoins"]


## QUESTIONS reformulation

##LONG ENTITIES ABBREVIATED

#PROBABLY NOT DOING THEM any mre





##SLANG

## INCISI

In [ ]:
sentences=["This, my friend, is epic "]
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

This, my friend, is epic 
['[ARG1: This , my friend] , [V: is] [ARG2: epic]']





## DIfferent frames or labels recognition

##With mutiple verbs, argoument confusion or shiet

##LONG SETNECES DEPEPNDENCY - coreference


## SAXON GENITIVES

In [ ]:
paired_sentences = [    ["The president's decision to withdraw troops from the region caused controversy among military leaders.",      "The decision of the president to withdraw troops from the region caused controversy among military leaders."],
    ["The CEO's success in turning around the struggling company was due in large part to her innovative marketing strategies.",      "The success of the CEO in turning around the struggling company was due in large part to her innovative marketing strategies."],
    ["The director's vision for the film was not fully realized due to budget constraints and scheduling conflicts.",      "The vision of the director for the film was not fully realized due to budget constraints and scheduling conflicts."],
    ["The professor's lecture on the history of ancient Rome was attended by a packed auditorium of eager students.",      "The lecture on the history of ancient Rome of the professor was attended by a packed auditorium of eager students."],
    ["The artist's latest work, a sculpture made entirely of recycled materials, was featured in a prominent gallery exhibition.",      "The latest work of the artist, a sculpture made entirely of recycled materials, was featured in a prominent gallery exhibition."],
    ["The athlete's rigorous training regimen and strict diet led to her record-breaking performance at the championship.",      "The rigorous training regimen and strict diet of the athlete led to her record-breaking performance at the championship."],
    ["The author's use of symbolism and metaphor in her novel added depth and complexity to the story.",      "The use of symbolism and metaphor in her novel of the author added depth and complexity to the story."]
]


##[PROLLY NOT]INTRANSITIVE VERBS

In [ ]:
sentences = ['The sun rises in the east.',
             'He slept for 10 hours.',
             'They danced all night.',
             'The flowers bloomed in the spring.',
             'The baby cried all night.',
             'The cat meowed loudly.',
             'The car stopped suddenly.',
             'She ran around the park.',
             'The plane took off at 9 am.',
             'The wind blew fiercely.',
             'The tree swayed in the wind.',
             'The river flows downstream.',
             'The coffee brewed in the pot.',
             'The music played softly in the background.',
             'The bird chirped early in the morning.',
             'The children laughed and played.',
             'The fire crackled in the fireplace.',
             'The leaves fell from the trees.',
             'The boat sailed across the ocean.',
             'The train arrived at the station on time.']


In [ ]:
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

The sun rises in the east.
['[ARG1: The sun] [V: rises] [ARGM-LOC: in the east] .']



He slept for 10 hours.
['[ARG0: He] [V: slept] [ARGM-TMP: for 10 hours] .']



They danced all night.
['[ARG0: They] [V: danced] [ARGM-TMP: all night] .']



The flowers bloomed in the spring.
['[ARG0: The flowers] [V: bloomed] [ARGM-TMP: in the spring] .']



The baby cried all night.
['[ARG0: The baby] [V: cried] [ARGM-TMP: all night] .']



The cat meowed loudly.
['[ARG0: The cat] [V: meowed] [ARGM-MNR: loudly] .']



The car stopped suddenly.
['[ARG1: The car] [V: stopped] [ARGM-MNR: suddenly] .']



She ran around the park.
['[ARG0: She] [V: ran] [ARGM-DIR: around the park] .']



The plane took off at 9 am.
['[ARG1: The plane] [V: took] off [ARGM-TMP: at 9 am] .']



The wind blew fiercely.
['[ARG1: The wind] [V: blew] [ARGM-MNR: fiercely] .']



The tree swayed in the wind.
['[ARG1: The tree] [V: swayed] [ARGM-LOC: in the wind] .']



The river flows downstream.
['[ARG1: The river] [V: flows] 

In [ ]:
sentences = ['The snow fell gently from the sky.',
             'She laughed uncontrollably at the joke.',
             'The sun set over the horizon.',
             'The children played happily in the park.',
             'The fire burned brightly in the fireplace.',
             'The wind howled outside the window.',
             'The cake baked for an hour.',
             'The door opened slowly.',
             'The crowd cheered loudly at the concert.',
             'The athlete ran around the track.',
             'The tree grew tall and strong.',
             'The river flowed peacefully.',
             'The bird flew gracefully through the air.',
             'The fish swam quickly in the water.',
             'The dog barked loudly at the mailman.',
             'The car drove smoothly down the road.',
             'The plane flew high above the clouds.',
             'The moon shone brightly in the sky.',
             'The stars twinkled in the night sky.',
             'The flowers swayed in the breeze.']


In [ ]:
for s in sentences:
  print(s)
  pred=predictor.predict(s)
  desc=[x['description'] for x in pred['verbs']]
  #print(verbs)
  print(desc)
  print("\n\n")

The snow fell gently from the sky.
['[ARG1: The snow] [V: fell] [ARGM-MNR: gently] [ARG3: from the sky] .']



She laughed uncontrollably at the joke.
['[ARG0: She] [V: laughed] [ARGM-MNR: uncontrollably] [ARG2: at the joke] .']



The sun set over the horizon.
['[ARG1: The sun] [V: set] [ARG2: over the horizon] .']



The children played happily in the park.
['[ARG0: The children] [V: played] [ARGM-MNR: happily] [ARGM-LOC: in the park] .']



The fire burned brightly in the fireplace.
['[ARG1: The fire] [V: burned] [ARGM-MNR: brightly] [ARGM-LOC: in the fireplace] .']



The wind howled outside the window.
['[ARG0: The wind] [V: howled] [ARGM-LOC: outside the window] .']



The cake baked for an hour.
['[ARG1: The cake] [V: baked] [ARGM-TMP: for an hour] .']



The door opened slowly.
['[ARG1: The door] [V: opened] [ARGM-MNR: slowly] .']



The crowd cheered loudly at the concert.
['[ARG0: The crowd] [V: cheered] [ARGM-MNR: loudly] [ARG1: at the concert] .']



The athlete ran around 

In [ ]:
sent="Many people keep falling for this CON - GAME that lower taxes on the rich benefits everyone . "
predictor.predict(sent)

{'verbs': [{'verb': 'keep',
   'description': '[ARG0: Many people] [V: keep] [ARG1: falling for this CON - GAME that lower taxes on the rich benefits everyone] .',
   'tags': ['B-ARG0',
    'I-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O']},
  {'verb': 'falling',
   'description': '[ARG1: Many people] keep [V: falling] [ARGM-PRP: for this CON - GAME that lower taxes on the rich benefits everyone] .',
   'tags': ['B-ARG1',
    'I-ARG1',
    'O',
    'B-V',
    'B-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'I-ARGM-PRP',
    'O']}],
 'words': ['Many',
  'people',
  'keep',
  'falling',
  'for',
  'this',
  'CON',
  '-',
  'GAME',
  'that',
  'lower'

In [ ]:
predictor.predict("He walked the dog to the park and back.")

{'verbs': [{'verb': 'walked',
   'description': '[ARG0: He] [V: walked] [ARG1: the dog] [ARGM-GOL: to the park and back] .',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'B-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'O']}],
 'words': ['He',
  'walked',
  'the',
  'dog',
  'to',
  'the',
  'park',
  'and',
  'back',
  '.']}

In [ ]:
predictor.predict("He walked with the dog to the park and back.")

{'verbs': [{'verb': 'walked',
   'description': '[ARG0: He] [V: walked] [ARGM-COM: with the dog] [ARGM-GOL: to the park and back] .',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARGM-COM',
    'I-ARGM-COM',
    'I-ARGM-COM',
    'B-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'I-ARGM-GOL',
    'O']}],
 'words': ['He',
  'walked',
  'with',
  'the',
  'dog',
  'to',
  'the',
  'park',
  'and',
  'back',
  '.']}

In [ ]:
predictor.predict("I see a man with a telescope under his arm")

{'verbs': [{'verb': 'see',
   'description': '[ARG0: I] [V: see] [ARG1: a man with a telescope under his arm]',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1']}],
 'words': ['I',
  'see',
  'a',
  'man',
  'with',
  'a',
  'telescope',
  'under',
  'his',
  'arm']}

In [ ]:
predictor.predict("The plane took a large turn")

{'verbs': [{'verb': 'took',
   'description': '[ARG0: The plane] [V: took] [ARG1: a large turn]',
   'tags': ['B-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'I-ARG1']}],
 'words': ['The', 'plane', 'took', 'a', 'large', 'turn']}